# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [418]:
import pandas as pd

In [419]:

data = pd.read_csv('/datasets/data.csv')
data.info()
print (data.head(20))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4

В начале открываем файл с данными, используя read_csv. Затем с помощью метода info() проводим общий обзор данных. Общее количество строк 21525, столбцов 12. Названия столбцов корректны. Нет необходимости переименовывать. Видны пропуски данных в столбцах days_employed и total_income. Оба столбца должны быть заполнены вещественными числами. Данные в столбце total_income могут отсутствовать в связи с тем, что не все заемщики их предоставили. Проанализируем на какие данные строки могут приходиться пропуски данных в столбце total_income. Для этого воспользуемся методом isna() и выведем на экран первый 20 строк с пропусками в столбце total_income.   

In [420]:
print(data[data['total_income'].isna()].head(10))

    children  days_employed  dob_years education  education_id  \
12         0            NaN         65   среднее             1   
26         0            NaN         41   среднее             1   
29         0            NaN         63   среднее             1   
41         0            NaN         50   среднее             1   
55         0            NaN         54   среднее             1   
65         0            NaN         21   среднее             1   
67         0            NaN         52    высшее             0   
72         1            NaN         32    высшее             0   
82         2            NaN         50    высшее             0   
83         0            NaN         52   среднее             1   

            family_status  family_status_id gender  income_type  debt  \
12       гражданский брак                 1      M    пенсионер     0   
26        женат / замужем                 0      M  госслужащий     0   
29  Не женат / не замужем                 4      F    

Обращает внимание, что: 
1. В случае пропуска в столбце total_income всегда пропущены данные и в столбце days_employed (после использования метода info () мы также знаем, что общее количество пропусков в обоих столбцах совпадает. 
2. В большинстве строк со значением NaN в столбце family_status значение 'женат / замужем' (18 из 30), а в графе gender значение  значение F (21 из 30). Используя условие и метод count() оценим какое количество строк с этими значениями и сравним с общим количеством строк в таблице.

In [421]:
print (data[data['family_status'] == 'женат / замужем']['family_status'].count())
print (data[data['gender'] == 'F']['gender'].count())

12380
14236


Из полученных значений видим, что пропорции в нашей выборке и пропорции в целом по датафрейму по значениям 'женат / замужем' и 'F' совпадают. Значит в дальнейшей работе этим фактом можно пренебречь.

### Шаг 2.1 Заполнение пропусков

Разберем ситуацию с пропусками в столбце 'total_income'. Не заполнено около 10% процентов значений. С ними можно поступить следующим образом:
1. Пренебречь, удалив из выборки.
2. Заменить на среднее арифметическое значение.
3. Заменить на медианное значение

Проанализируем уникальные значения в столбце 'income_type', используя метод unique()

In [422]:
print (data['income_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


In [423]:
print (data[data['income_type'] == 'безработный']['income_type'].count())

2


Видим, что безработных в базе только два человека. Значит отсутствие данных о количестве дней трудового стажа. Значит причина отсутствия данных в том, что они просто не были предоставлены клиентами в процессе заполнения анкет. Для заполнения  ячеек предпочтительно использовать медианное значение, так как наибольшее количество значений лежит в дипазоне от 100 000 до 250 000 (видно из выведенного фрагмента). Поэтому использование среднего арифметического значительно сместит в нижнюю стороны значения в незаполненных ячейках.

In [424]:
income_avg = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(value=income_avg)


### Шаг 2.2 Проверка данных на аномалии и исправления.

Проанализируем значения и пропуски в столбце 'days_employed'. Речь идет об общем трудовом стаже в днях. Количество рабочих дней в году примем за 250 (например, в 2022, в соответствии с производственным календарем, их 247). Примем, что в обычном случае клиент может работать максиму с 18 лет до 65. То есть 47 лет. А это значит максимально 11 609. Все заметно большие значения являются аномальными. Оценим их количество в нашей базе, используя метод count() и условие.

In [425]:
print (data[data['days_employed'] > 11609]['days_employed'].count())

3445


Видим, что таких аномальных значений у нас в базе 3445. Предположительно это может быть связано с некорректным занесением в базу. Второй тип аномалий - это отрицательное количество дней трудового стажа. Незаполненых ячеек у нас 2174, а анамально больших 3455. Выдвигаем гипотезу, что остальные поля заполнены отрицательными значениями. Т.е. их должно быть 15 906. Вновь используем условие и метод count().


In [426]:
print (data[data['days_employed'] < 0]['days_employed'].count())

15906


Наша гипотеза подтверждена. Причина скорее всего в неверном заполнении полей форм. Для исправления значений выполним следующую последовательность действий:
1. Заменим все значения на значения по модулю, используя стандартную функцию abs()
2. Заменим аномально большие значения на максимально возможное количество дней стажа, которое мы рассчитали.
3. Заменим все пропущенные значения на медианные по столбцу.

In [427]:
data['days_employed']=abs(data['days_employed'])
data.loc[data['days_employed'] > 11609,'days_employed'] = 11609

days_employed_avg = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(value=days_employed_avg)



Проанализируем уникальные значения в столбце 'dob_years', используя методы unique(), а затем count(). Видим, что в столбце встречается значение 0. Такого возраста быть не может. Скорее всего имеем дело с ошибкой заполнения. Видим, что таких строк 101. Поэтому применим замену на медианное значение.

In [428]:
print(data['dob_years'].unique())
print(data.groupby('dob_years')['debt'].count())

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
dob_years
0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: debt, dtype: int64


In [429]:
income_avg = data['dob_years'].median()
data['dob_years'] = data['dob_years'].replace(0, income_avg).astype(int)

Исследуя столбец 'gender' с помощью методов unique(), а затем count(), находим в одной из строк значение 'XNA'. Так как такая строка только одна, а в РФ по-прежнему официально признаны только два гендера, то строку с неизвестным гендером просто удалим. Для этого сначала заменим значение 'XNA' на пропуск, а затем удалим эту строку используя метод dropna() 

In [430]:
print(data['gender'].unique())
print(data.groupby('gender')['debt'].count())

['F' 'M' 'XNA']
gender
F      14236
M       7288
XNA        1
Name: debt, dtype: int64


In [431]:
data.loc[data['gender'] == 'XNA','gender'] = None
data = data.dropna().reset_index(drop=True) 
print(data['gender'].unique())
print(data.groupby('gender')['debt'].count())

['F' 'M']
gender
F    14236
M     7288
Name: debt, dtype: int64


### Шаг 2.3. Изменение типов данных.

In [432]:
data['total_income'] = data['total_income'].astype('int')


Выше мы заменили тип данных в столбце 'total_income' на int, используя метод astype()

### Шаг 2.4. Удаление дубликатов.

In [433]:
data ['education'] = data ['education'].str.lower()

data  =data.drop_duplicates()

Выше мы убрали дубликаты из таблицы, используя метод data.duplicated() для их подсчета и метод drop_duplicates(). Затем мы удалили неявные дубликаты по столбцу 'education'. Использовали метод str.lower(), чтобы все содержимое было в одном регистре, а затем повторили связку duplicated().count()) и drop_duplicates()

Проведем аналогичный поиск по столбцам 'education', 'family_status', 'income_type'.

In [434]:
print(data['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [435]:
print(data['family_status'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


In [436]:
print(data['income_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Выше с использованием метода unique() мы определили, что в столбцах 'education', 'family_status', 'income_type'нет аномальных и дублирующихся значений 

In [437]:
print(data['children'].unique())

[ 1  0  3  2 -1  4 20  5]


Видим, что в количестве детей есть аномальные значения: отрицательные или слишком большие. И то, и другое, скорее всего ошибка ввода. Поэтому заменим отрицательные значения на модуль с помощью метода abs(). А большие на медианные, используя median().

In [438]:
data['children']=abs(data['children'])

Используя groupby, определяем, что на аномально большие значения приходится менее одного процента данных (ниже).

In [439]:
print(data.groupby('children').count())

          days_employed  dob_years  education  education_id  family_status  \
children                                                                     
0                 14089      14089      14089         14089          14089   
1                  4855       4855       4855          4855           4855   
2                  2052       2052       2052          2052           2052   
3                   330        330        330           330            330   
4                    41         41         41            41             41   
5                     9          9          9             9              9   
20                   76         76         76            76             76   

          family_status_id  gender  income_type   debt  total_income  purpose  
children                                                                       
0                    14089   14089        14089  14089         14089    14089  
1                     4855    4855         4855   4855   

Мы можем либо вообще удалить эти строки, либо заменить в них значение в столбце 'children' на среднее арифметическое. Но удаление может исказить результаты исследования, если это заемщики из одной группы. Поэтому заменяем на среднее арифметическое и проверяем полученный результат с помощью groupby

In [440]:
children_avg=data['children'].mean().astype('int')
print (children_avg)
data.loc[data['children'] == 20,'children'] = children_avg
print(data.groupby('children').count())

0
          days_employed  dob_years  education  education_id  family_status  \
children                                                                     
0                 14165      14165      14165         14165          14165   
1                  4855       4855       4855          4855           4855   
2                  2052       2052       2052          2052           2052   
3                   330        330        330           330            330   
4                    41         41         41            41             41   
5                     9          9          9             9              9   

          family_status_id  gender  income_type   debt  total_income  purpose  
children                                                                       
0                    14165   14165        14165  14165         14165    14165  
1                     4855    4855         4855   4855          4855     4855  
2                     2052    2052         2052   205

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма.

In [441]:
education_dict = data[['education_id', 'education']]
family_status_dict = data[['family_status_id', 'family_status']]
print (education_dict.head(10))
print (family_status_dict.head(10))                      

   education_id education
0             0    высшее
1             1   среднее
2             1   среднее
3             1   среднее
4             1   среднее
5             0    высшее
6             0    высшее
7             1   среднее
8             0    высшее
9             1   среднее
   family_status_id     family_status
0                 0   женат / замужем
1                 0   женат / замужем
2                 0   женат / замужем
3                 0   женат / замужем
4                 1  гражданский брак
5                 1  гражданский брак
6                 0   женат / замужем
7                 0   женат / замужем
8                 1  гражданский брак
9                 0   женат / замужем


Строки в полученных датафреймах многократно дублируются. Уберем дубликаты, используя метод drop_duplicates(). 

In [442]:
education_dict=education_dict.drop_duplicates()
family_status_dict=family_status_dict.drop_duplicates()

print (education_dict.head(30))
print (family_status_dict.head(30))

      education_id            education
0                0               высшее
1                1              среднее
13               2  неоконченное высшее
31               3            начальное
2963             4       ученая степень
    family_status_id          family_status
0                  0        женат / замужем
4                  1       гражданский брак
18                 2         вдовец / вдова
19                 3              в разводе
24                 4  Не женат / не замужем


Удалим из исходного датафрейма столбцы 'education' и 'family_status', используя метод pop()

In [443]:
data.pop('education')
data.pop('family_status')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21452 entries, 0 to 21523
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21452 non-null  int64  
 1   days_employed     21452 non-null  float64
 2   dob_years         21452 non-null  int64  
 3   education_id      21452 non-null  int64  
 4   family_status_id  21452 non-null  int64  
 5   gender            21452 non-null  object 
 6   income_type       21452 non-null  object 
 7   debt              21452 non-null  int64  
 8   total_income      21452 non-null  int64  
 9   purpose           21452 non-null  object 
dtypes: float64(1), int64(6), object(3)
memory usage: 1.8+ MB


### Шаг 2.6. Категоризация дохода.

Для категоризации доходов мы напишем функцию, куда в качестве значения будет передаваться доход (значение 'total_income'). И с помощью метода apply применим ее к столбцу 'total_icome', а значения поместим в столбец 'total_income_category'

In [444]:

def tot_inc_cat(t_incom):
    if (t_incom >0) & (t_incom<30000):
        return 'E'
    elif (t_incom< 50000) & (t_incom>=30001):
        return 'D'
    elif (t_incom>=50001)  & (t_incom< 200000):
        return 'C'
    elif (t_incom>=200001) & (t_incom< 1000000):
        return 'B'
    else:
        return 'A'
data['total_income_category']=data['total_income'].apply(tot_inc_cat)

print (data.head(10))

   children  days_employed  dob_years  education_id  family_status_id gender  \
0         1    8437.673028         42             0                 0      F   
1         1    4024.803754         36             1                 0      F   
2         0    5623.422610         33             1                 0      M   
3         3    4124.747207         32             1                 0      M   
4         0   11609.000000         53             1                 1      F   
5         0     926.185831         27             0                 1      M   
6         0    2879.202052         43             0                 0      F   
7         0     152.779569         50             1                 0      M   
8         2    6929.865299         35             0                 1      F   
9         0    2188.756445         41             1                 0      M   

  income_type  debt  total_income                     purpose  \
0   сотрудник     0        253875               покупк

### Шаг 2.7. Категоризация целей кредита.

Для категоризации целей получения кредита посмотрим перечень уникальных значений в столбце 'purpose' c помощью метода unique() и попробуем увидеть закономерности.

In [445]:
print (data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Видим, что:
1. В описании всех целей, связанных с автомобилями присутствует "авто"
2. В описании всех целей, связанных с образованием присутствет "образовани"
3. В описании всех целей, связанных со свадьбами присутствует "свадьб"

In [446]:
def purp_cat_name(purpose_name):
    try:
        
        if 'авто' in purpose_name:
            return 'операции с автомобилем'
        elif 'образовани' in purpose_name:
            return 'получение образования'
        elif 'свадьб'in purpose_name:
            return 'проведение свадьбы'
        else:
            return 'операции с недвижимостью'
    except:
        return 'некорректное значение'
    
data['purpose_category']=data['purpose'].apply(purp_cat_name)


In [447]:
print (data.head(10))

   children  days_employed  dob_years  education_id  family_status_id gender  \
0         1    8437.673028         42             0                 0      F   
1         1    4024.803754         36             1                 0      F   
2         0    5623.422610         33             1                 0      M   
3         3    4124.747207         32             1                 0      M   
4         0   11609.000000         53             1                 1      F   
5         0     926.185831         27             0                 1      M   
6         0    2879.202052         43             0                 0      F   
7         0     152.779569         50             1                 0      M   
8         2    6929.865299         35             0                 1      F   
9         0    2188.756445         41             1                 0      M   

  income_type  debt  total_income                     purpose  \
0   сотрудник     0        253875               покупк

Для определения наличия зависимости между доходом заемщика и вероятностью возврата кредита построим сводную таблицу. В качестве результата сумма по столбцу debt. Так значения там только 0 (нет задолженности) или 1 (имеется), то сумма хорошо ответит на нужный вопрос.

In [448]:
data_pivot_income = data.pivot_table(index=['total_income_category'], values='debt', aggfunc={'sum', 'count'})


data_pivot_income['ratio_per']=data_pivot_income['sum']/data_pivot_income['count']*100
print(data_pivot_income)    

                       count   sum  ratio_per
total_income_category                        
A                         25     2   8.000000
B                       5040   356   7.063492
C                      16015  1360   8.492039
D                        350    21   6.000000
E                         22     2   9.090909


Для определения наличия зависимости между количеством детей у заемщика и вероятностью возврата кредита построим сводную таблицу. В качестве результата сумма по столбцу debt. 

In [449]:
data_pivot_children = data.pivot_table(index=['children'], values='debt', aggfunc={'sum', 'count'})


data_pivot_children['ratio_per']=data_pivot_children['sum']/data_pivot_children['count']*100
print(data_pivot_children)


          count   sum  ratio_per
children                        
0         14165  1071   7.560890
1          4855   445   9.165808
2          2052   194   9.454191
3           330    27   8.181818
4            41     4   9.756098
5             9     0   0.000000


Для определения наличия зависимости между семейным статусом заемщика и вероятностью возврата кредита построим сводную таблицу. В качестве результата сумма по столбцу debt. После это присоединим к ней датафрейм family_status_dict, где содержатся описания семейных статусов

In [450]:
data_pivot_family = data.pivot_table(index=['family_status_id'], values='debt', aggfunc={'sum', 'count'})


data_pivot_family['ratio']=data_pivot_family['sum']/data_pivot_family['count']*100

data_pivot_family_f = data_pivot_family.merge(family_status_dict, on='family_status_id', how='right')
 
print(data_pivot_family_f)        


   family_status_id  count  sum     ratio          family_status
0                 0  12339  931  7.545182        женат / замужем
1                 1   4149  388  9.351651       гражданский брак
2                 2    959   63  6.569343         вдовец / вдова
3                 3   1195   85  7.112971              в разводе
4                 4   2810  274  9.750890  Не женат / не замужем


Для определения наличия зависимости между целью кредита и вероятностью возврата кредита построим сводную таблицу. В качестве результата сумма по столбцу debt. После это присоединим к ней датафрейм family_status_dict, где содержатся описания семейных статусов

In [451]:
data_pivot_purpose_category = data.pivot_table(index=['purpose_category'], values='debt', aggfunc={'sum', 'count'})

data_pivot_purpose_category['ratio']=data_pivot_purpose_category['sum']/data_pivot_purpose_category['count']*100
print(data_pivot_purpose_category)

                          count  sum     ratio
purpose_category                              
операции с автомобилем     4306  403  9.359034
операции с недвижимостью  10810  782  7.234043
получение образования      4013  370  9.220035
проведение свадьбы         2323  186  8.006888


### Ответы на вопросы.

##### Вывод 1:

О взаимосвязи количества детей и возврата кредита. Из сводной таблицы мы видим, что величина невозвратов колеблется в диапазоне от 7,56% до 9,75%. Исключение составляют случаи, когда у заемщика пять детей. Там все кредиты вернулись. В остальных случаях четкой закономерности не видно. Но нужно отметить, что для заемщиков без детей вероятность невозврата лежит на нижней границе диапазона. Риск немного, но ниже.

##### Вопрос 2:

Исследование взаимосвязи между семейным статусом заемщика и вероятностью непогашения кредита приводит нас к следующим выводам:
     - разброс значений в диапаозоне от 6,57% до 9,76%. Хотя разница почти в полтора раза, но в абсолютном измерении разброс незначительный
     - вероятность невозврата кредита несколько выше для тех, кто имеет статус "гражданский брак" и "не женат"

##### Вопрос 3:

Исследуя аналогично и зависимость возврата кредита от категории дохода заемщика, мы приходим к схожим выводам. Разброс вероятности невозврата находится в диапазоне от 6% до 9,1%. Максимальное количество невозвратов приходится на заемщиков группы Е, т.е. минимальные доходы. Но резко снижается в следующей группе. Четкой тенденции не видно. Но минимальная вероятность невозврата приходится на группу D. С доходами от 31 до 50 рублей.

##### Вопрос 4:

Анализ взаимосвязи между целью кредита и возможностью его невозврата лежит в еще более узком диапазоне. От 7,23% до 9,35%. Можно сказать, что вероятность возврата не зависит от целей кредита.

## Общий вывод:

Вероятность возврата кредита на основании проведенного исследования не зависит от дохода, семейного статуса и количества детей у заемщика. Также и от целей займа. Риск невозврата несколько выше для заемщиков со статусом 'Не женат / не замужем', а также когда заемщики со статусом 'гражданский брак' обращаются за займом на свадьбу.

Целью настоящего исследования было установить влияние семейного статуса, количества детей и доходов заемщика на вероятность возврата кредита.
В предоставленных данных потребовалось провести предварительную обработку для их использования. Отсутстовали данные о доходах примерно в 10% случаев. Данные были заполнены медианныыми значениями и это не должно было значительно исказить выводы исследования.
Также примерно в 15% случаев было некорректно заполнено поле с трудовым стажем заемщиков. Были внесены замены на медианные значения. Но важно отметить, что в рамках исследования зависимость вероятности возврата от возраста не рассматривалась.
В целом результ исследования показал отсутствие критических зависимостей вероятности возврата от семейного статуса, количества детей, уровня доходов и цели кредита. Все значения лежат в достаточно узком диапазоне от 6% до 9,75%.

Риск невозвра несколько выше в случаях, когда берется кредит на операции с автомобилями и составляет 9,35%. На эту группу заемщиков стоит обратить большее внимание.